In [1]:
import pandas as pd

weibo_df = pd.read_excel('weibo_data.xlsx')
corpus = []
for i in range(1,len(weibo_df)):
    corpus.append(weibo_df.iloc[i,3])

#### TF-IDF + 传统机器学习（SVM, Random Forest）适用于简单分类任务

In [3]:
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# 示例数据
corpus = [
    "这部电影很好看，我非常喜欢。",
    "剧情很感人，演员演技也很棒。",
    "太无聊了，浪费时间。",
    "这是一部烂片，后悔看了。",
    "非常精彩的故事，推荐观看。",
    "不值得看，情节太差了。"
]

# 自己打标签 - 现在有一些好的训练集是自己带标签的
labels = [1, 1, 0, 0, 1, 0]  # 1 代表好评，0 代表差评

# 中文分词
corpus_text = [' '.join(jieba.cut(text)) for text in corpus]

# tf-idf(list)
vectorizer = TfidfVectorizer()  # list
X = vectorizer.fit_transform(corpus_text)

# 训练 SVM 分类器 - 适用于二分类
svm_model = SVC(kernel = 'linear')
svm_model.fit(X,labels)

# 训练 Random Forest 分类器 - 适用于二分类和多分类
rf_model = RandomForestClassifier(n_estimators = 100,
                                      max_depth = 10,
                                      random_state = 42)
rf_model.fit(X,labels)

# prediction
test_text = '这部电影真的太棒了！'
text_vector = vectorizer.transform([' '.join(jieba.cut(test_text))])
print('svm_model_prediction: ',svm_model.predict(text_vector))
print('rf_model_prediction: ',rf_model.predict(text_vector))

svm_model_prediction:  [1]
rf_model_prediction:  [0]


#### Word2Vec 可提供更丰富的语义信息，在低资源条件下表现良好。

In [5]:
from gensim.models import Word2Vec
import numpy as np

### ---Word2Vec 模型训练---

# 准备输入格式： 
# corpus_cut_list 将原始文本列表转换成 词语列表的列表（即 [['这', '部', '电影', '很', '好看'], ['剧情', '很', '感人'], ...]）。这是 Word2Vec 模型要求的输入格式。
corpus_cut_list = [list(jieba.cut(text)) for text in corpus]

# 训练Word2Vec模型
word2vec_model = Word2Vec(sentences = corpus_cut_list, # 传入训练数据
                          vector_size = 100, # 设置每个词语将被表示成一个 100 维的向量（词向量的维度）
                          window = 5, # 设定模型在训练时会考虑一个词语前后 5 个词语的上下文
                          min_count = 1, # 要求词语至少出现 1 次才会被纳入词汇表
                          workers = 4) # 使用 4 个 CPU 核心进行并行训练，加速过程
# 训练完成后，word2vec_model 内部存储了每个词语（如 '电影', '好看', '无聊'）对应的 100 维数值向量。

### ---定义句子向量计算函数---

# 计算句子向量
# Word2Vec 直接生成词语的向量，而我们的分类器需要整个句子的向量。这个函数实现了从词向量到句子向量的转换
def get_sentence_vector(sentence,model):
    # 1.分词： 对输入句子进行分词。
    words = list(jieba.cut(sentence))
    
    # 2.提取词向量： 遍历分词结果，从 Word2Vec 模型的词汇表 (model.wv) 中提取每个词对应的 100 维向量。
    vectors = [model.wv[word] for word in words if word in model.wv]
    # model.wv 是 Gensim Word2Vec 模型的一个关键属性，它代表 Word Vectors（词向量）。
    # model.wv['电影'] -> 返回一个 100 维 NumPy 数组
    
    # 3.平均求和 (Averaging)： 将句子中所有词的词向量进行求平均。这个平均向量（也是 100 维）就被用来代表整个句子的语义特征。
    # 注意： 如果句子为空或所有词都不在词汇表中，则返回一个 100 维的零向量。
    return np.mean(vectors,axis = 0) if vectors else np.zeros(100)

### ---训练数据特征转换与模型训练---

# 单一的、统一的二维 NumPy 矩阵
# 最外层的 np.array() 的作用是：将列表推导式生成的一系列独立的 100 维 NumPy 数组（即每个句子的向量），聚合 成一个单一的、统一的二维 NumPy 矩阵，以供 Scikit-learn 分类器 (SVM, Random Forest) 使用。
X_train_vectors = np.array([get_sentence_vector(text,word2vec_model) for text in corpus])

# 训练 SVM
svm_model.fit(X_train_vectors, labels)
# 训练随机森林
rf_model.fit(X_train_vectors, labels)


# 预测
test_vector = get_sentence_vector(test_text, word2vec_model).reshape(1, -1)
# .reshape(1, -1) 是必要的，因为它将单个向量从 $(100,)$ 形状调整为 $(1, 100)$，以符合 predict 函数对输入格式的要求
print("SVM 预测结果:", svm_model.predict(test_vector))
print("Random Forest 预测结果:", rf_model.predict(test_vector))

SVM 预测结果: [1]
Random Forest 预测结果: [1]


#### BERT 适用于更复杂的 NLP 任务，能够理解更深层次的语义关系。

In [10]:
from transformers import BertTokenizer, BertModel
import torch

# 加载中文 BERT
tokenizer = BertTokenizer.from_pretrained('hfl/chinese-bert-wwm')
bert_model = BertModel.from_pretrained('hfl/chinese-bert-wwm')

# 获取句子 BERT 向量
def get_bert_embedding(text):
    inputs = tokenizer(text,
                       return_tensors = 'pt',
                       padding = True,
                       truncation = True,
                       max_length = 512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # 取 [CLS] token 的向量

# 计算训练集向量
X_train_bert = np.array([get_bert_embedding(text) for text in corpus])

# 训练 SVM
svm_model.fit(X_train_bert,labels)
# 训练随机森林
rf_model.fit(X_train_bert, labels)

# perdict
test_vector = get_bert_embedding(test_text).reshape(1,-1)
# get_bert_embedding(test_text) 的原始输出：这个函数返回的是单个句子的嵌入向量，它是一个 一维 NumPy 数组，形状为 $(768,)$。
# predict() 函数的要求：Scikit-learn 的 predict() 函数始终期望其输入是一个二维数组，即使你只预测一个样本。它要求输入形状为 $(\text{n\_samples}, \text{n\_features})$。
# reshape(1, -1) 的作用：它将原始的 $(768,)$ 一维数组 转换成一个 二维数组，形状变为 $(1, 768)$。
#     这里的 $1$ 代表 1 个样本（即您的测试文本）。
#     这里的 $-1$ 是一个通配符，它告诉 NumPy 自动计算出该维度的大小，即 $768$ 个特征。
print("SVM 预测结果:", svm_model.predict(test_vector))
print("Random Forest 预测结果:", rf_model.predict(test_vector))

/opt/anaconda3/envs/py39/lib/python3.9/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at hfl/chinese-bert-wwm were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a

SVM 预测结果: [1]
Random Forest 预测结果: [1]
